In [1]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

## Loading Data Incendie


In [2]:
df_incendie= pd.read_csv('./Data/Incendies.csv', sep=';', header=3)
df_incendie.shape

(27785, 24)

In [3]:
df_incendie.columns

Index(['Année', 'Numéro', 'Département', 'Code INSEE', 'Nom de la commune',
       'Date de première alerte', 'Surface parcourue (m2)',
       'Surface forêt (m2)', 'Surface maquis garrigues (m2)',
       'Autres surfaces naturelles hors forêt (m2)', 'Surfaces agricoles (m2)',
       'Autres surfaces (m2)', 'Surface autres terres boisées (m2)',
       'Surfaces non boisées naturelles (m2)',
       'Surfaces non boisées artificialisées (m2)',
       'Surfaces non boisées (m2)', 'Précision des surfaces',
       'Type de peuplement', 'Nature', 'Décès ou bâtiments touchés',
       'Nombre de décès', 'Nombre de bâtiments totalement détruits',
       'Nombre de bâtiments partiellement détruits', 'Précision de la donnée'],
      dtype='object')

In [4]:
colonnes_utiles = [
    'Année',
    'Numéro',
    'Département',
    'Code INSEE',
    'Date de première alerte',
    'Surface parcourue (m2)',  
    'Surface forêt (m2)',
    'Surface autres terres boisées (m2)',
    'Surfaces non boisées naturelles (m2)',
    'Surfaces non boisées artificialisées (m2)',
    'Surfaces non boisées (m2)'
]

incendie = df_incendie[colonnes_utiles].copy()


In [5]:
incendie.head(1)

,Année,Numéro,Département,Code INSEE,Date de première alerte,Surface parcourue (m2),Surface forêt (m2),Surface autres terres boisées (m2),Surfaces non boisées naturelles (m2),Surfaces non boisées artificialisées (m2),Surfaces non boisées (m2)
0,2015,5769,66,66100,2015-01-01 00:09:00,5000,5000.0,NaN,NaN,NaN,NaN


## Cleaning Data Incendie

In [6]:
incendie.isna().any()

Année                                        False
Numéro                                       False
Département                                  False
Code INSEE                                   False
Date de première alerte                      False
Surface parcourue (m2)                       False
Surface forêt (m2)                            True
Surface autres terres boisées (m2)            True
Surfaces non boisées naturelles (m2)          True
Surfaces non boisées artificialisées (m2)     True
Surfaces non boisées (m2)                     True
dtype: bool

In [7]:
incendie.fillna(0, inplace=True)

In [8]:
# On fusionne les surfaces forêt + autres terres boisées, et les surfaces non boisées naturelles + artificialisées, pour créer des indicateurs synthétiques de type de terrain
incendie.iloc[:,6] = incendie.iloc[:,6] + incendie.iloc[:,7]
incendie.iloc[:,9] = incendie.iloc[:,9] + incendie.iloc[:,10]

In [9]:
# Suppression des colonnes redondantes après fusion manuelle des surfaces boisées et non boisées
incendie.drop(['Surface autres terres boisées (m2)','Surfaces non boisées (m2)'], axis=1, inplace=True)

In [10]:
incendie.rename(columns={
    'Année': 'annee',
    'Numéro': 'id_inc',
    'Département': 'dep',
    'Code INSEE': 'code_insee',
    'Date de première alerte': 'date',
    'Surface parcourue (m2)': 'surf_brulee',
    'Surface forêt (m2)': 'surf_foret',
    'Surface autres terres boisées (m2)': 'surf_autres_bois',
    'Surfaces non boisées naturelles (m2)': 'surf_nb_naturelle',
    'Surfaces non boisées artificialisées (m2)': 'surf_nb_artificielle',
    'Surfaces non boisées (m2)': 'surf_nb_totale'
}, inplace=True)

In [11]:
incendie.shape

(27785, 9)

In [12]:
incendie.drop_duplicates(['date','id_inc']).shape

(27785, 9)

In [13]:
incendie.drop_duplicates(['code_insee','id_inc']).shape

(27756, 9)

In [14]:
incendie.dtypes

annee                     int64
id_inc                    int64
dep                      object
code_insee               object
date                     object
surf_brulee               int64
surf_foret              float64
surf_nb_naturelle       float64
surf_nb_artificielle    float64
dtype: object

In [15]:
# On enlève les départements corses
incendie = incendie[~incendie['code_insee'].astype(str).str.startswith(('2A', '2B'))]

In [16]:
# On convertit la colonne code_insee en entier
incendie['code_insee'] = incendie['code_insee'].astype(int)

In [17]:
# Regroupement des départements d’Île-de-France sous le code de Paris ('75'), comme dans les données IGN sauf pour la Seine-et-Marne 
incendie.loc[
    (incendie.dep == '75') |
    (incendie.dep == '91') |
    (incendie.dep == '92') |
    (incendie.dep == '93') |
    (incendie.dep == '94') |
    (incendie.dep == '95') |
    (incendie.dep == '78'),
    'dep'
] = '75'

In [18]:
# Convertir les dates en datetime
incendie['date'] = incendie.date.apply(lambda _: dt.datetime.strptime(_,"%Y-%m-%d %H:%M:%S"))

## Loading Geographic Data

In [19]:
coords= gpd.read_file('./Data/commune.shp')

In [20]:
# Affichage du nombre de communes et des colonnes disponibles dans le shapefile AdminExpress
coords.shape, coords.columns

((34935, 12),
 Index(['ID', 'NOM', 'NOM_M', 'INSEE_COM', 'STATUT', 'POPULATION', 'INSEE_CAN',
        'INSEE_ARR', 'INSEE_DEP', 'INSEE_REG', 'SIREN_EPCI', 'geometry'],
       dtype='object'))

In [21]:
coords = coords[['INSEE_COM', 'INSEE_DEP', 'geometry']].copy()
coords.columns = ['code_insee', 'dep', 'geometry']

In [22]:
# On sélectionne les colonnes utiles et on harmonise le code INSEE au format chaîne de 5 caractères
coords['code_insee'] = coords['code_insee'].astype(str).str.zfill(5)

In [23]:
coords.loc[(coords.dep=='75')|(coords.dep=='91')|(coords.dep=='92')|(coords.dep=='93')|(coords.dep=='94')|(coords.dep=='95')|(coords.dep=='78'), 'dep'] = '75'

In [24]:
# Harmonisation des code_insee dans le dataset incendie
incendie['code_insee'] = incendie['code_insee'].astype(str).str.zfill(5)

# Affichage des communes non matchées
incendie[~incendie['code_insee'].isin(coords['code_insee'])]


,annee,id_inc,dep,code_insee,date,surf_brulee,surf_foret,surf_nb_naturelle,surf_nb_artificielle
118,2015,6934,48,48122,2015-02-18 14:30:00,12500,6000.0,0.0,6500.0
189,2015,11315,48,48162,2015-03-07 21:30:00,10005,0.0,0.0,0.0
201,2015,10983,48,48022,2015-03-08 15:40:00,114600,0.0,0.0,0.0
212,2015,10961,48,48022,2015-03-09 13:02:00,138000,0.0,0.0,0.0
233,2015,10818,11,11097,2015-03-10 15:53:00,2000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
22174,2022,2504,71,71492,2022-07-31 13:10:00,5000,5000.0,0.0,0.0
22237,2022,17749,05,05141,2022-08-02 09:23:00,13130,1100.0,0.0,11800.0
22361,2022,17967,48,48184,2022-08-04 15:45:00,90000,0.0,0.0,0.0
22702,2022,18092,48,48154,2022-08-10 14:13:00,150,0.0,0.0,0.0


In [25]:
# Nombre de communes sans correspondance géographique
len(incendie[~incendie.code_insee.isin(coords.code_insee)])

190

In [26]:
# Date de la première alerte
incendie.date.dt.year.min()

2015

## Loading Data Forêt

In [27]:
arbre = pd.read_csv("./Data/COUVERT.csv", sep=";", low_memory=False)

In [28]:
arbre.shape, arbre.columns

((750822, 9),
 Index(['CAMPAGNE', 'IDP', 'STRATE', 'ESPAR_C', 'TCA', 'TCL', 'P1525', 'P7ARES',
        'Unnamed: 8'],
       dtype='object'))

In [ ]:
arbre.sort_values(['IDP','TCA'], inplace=True)

In [30]:
ign = arbre[['IDP', 'ESPAR_C']].drop_duplicates('IDP', keep='last').copy()

In [31]:
ign.columns = ['idp', 'essence']
ign.shape

(110459, 2)

In [32]:
ign.set_index('idp', inplace=True)

## Cleaning Data Forêt

In [34]:
ign.reset_index(inplace=True)
ign.isna().any()

index      False
idp        False
essence    False
dtype: bool

In [35]:
# valeurs NaN venant d'un dataset original?
arbre.isna().any()

CAMPAGNE      False
IDP           False
STRATE        False
ESPAR_C       False
TCA           False
TCL           False
P1525          True
P7ARES         True
Unnamed: 8     True
dtype: bool

## Loading Data Météo

In [65]:
station= pd.read_csv('./Data/meteo_station.txt', sep=';')

In [66]:
station.shape[0] - len(station.ID.drop_duplicates())

0

In [67]:
station.head(1)

,ID,Nom,Latitude,Longitude,Altitude
0,7005,ABBEVILLE,50.136,1.834,69


In [68]:
station.drop(['Nom', 'Altitude'], axis=1, inplace=True)

In [69]:
station.columns = station.columns.str.lower()

In [70]:
geometry = gpd.points_from_xy(station.longitude, station.latitude)
crs = 'epsg:4326'

stations = gpd.GeoDataFrame(station, geometry=geometry, crs=crs)

In [71]:
# préparations pour la prochaine exécution: conversion des projections
stations.geometry = stations.geometry.to_crs('epsg:2154')
coords.geometry = coords.geometry.to_crs('epsg:2154')

In [72]:
# test de proximité entre Point(Stations) et Polygone(Coords_fr)
stations = gpd.sjoin_nearest(coords, stations)
stations.head(1)

,code_insee,dep,geometry,index_right,id,latitude,longitude
0,29067,29,"POLYGON ((213580.300 6849033.200, 213591.600 6...",7,7117,48.825833,-3.473167


In [73]:
# récupération des codes départementaux via le code INSEE
stations['dep_insee'] = stations.code_insee.apply(lambda _: _[:2])

In [74]:
# changement pour les départements d'Île-de-France en 75
stations.loc[(stations.dep_insee=='75')|(stations.dep_insee=='91')|(stations.dep_insee=='92')|(stations.dep_insee=='93')|(stations.dep_insee=='94')|(stations.dep_insee=='95')|(stations.dep_insee=='78'), 'dep_insee'] = '75'

In [75]:
# différence entre "dep" et le département du code INSEE?
stations[stations.dep_insee!=stations.dep.astype(str)]

,code_insee,dep,geometry,index_right,id,latitude,longitude,dep_insee
34806,97110,971,"MULTIPOLYGON (((-6222575.570 6081172.109, -622...",52,78890,16.3350,-61.004000,97
34807,97114,971,"MULTIPOLYGON (((-6281930.798 6110794.953, -628...",54,78897,16.2640,-61.516333,97
34808,97119,971,"POLYGON ((-6238971.150 6123966.423, -6238971.1...",54,78897,16.2640,-61.516333,97
34809,97124,971,"POLYGON ((-6305588.814 6112702.415, -6305587.2...",54,78897,16.2640,-61.516333,97
34810,97109,971,"POLYGON ((-6297024.531 6111938.047, -6297046.5...",54,78897,16.2640,-61.516333,97
...,...,...,...,...,...,...,...,...
34921,97306,973,"MULTIPOLYGON (((-6541669.393 4413796.880, -654...",57,81401,5.4855,-54.031667,97
34922,97311,973,"POLYGON ((-6542090.338 4414746.650, -6541963.7...",57,81401,5.4855,-54.031667,97
34923,97303,973,"POLYGON ((-6455640.630 4331651.463, -6455640.3...",57,81401,5.4855,-54.031667,97
34927,97360,973,"POLYGON ((-6595005.257 4424713.565, -6595188.9...",57,81401,5.4855,-54.031667,97


In [76]:
# réorganisation du df Stations
stations.drop(['latitude', 'longitude', 'index_right','geometry'], axis=1, inplace=True)
stations.insert(2,'id',stations.pop('id'))

In [80]:
stations.head(1)

,code_insee,dep,id,dep_insee
0,29067,29,7117,29


In [81]:
# raccord des id.Stations à Incendie
incendie = incendie.merge(stations[['code_insee','id']], on='code_insee', how='left')

## Météo

In [91]:
meteo_ = pd.read_csv('./Data/meteo.csv')
meteo_.shape

(14811, 1)